In [1]:
import numpy as np
import time
import copy
inputs = []
file = open("inputs\\iH.txt",'r')
lines = file.readlines()
for line in lines:
    inputs.append(line)
v = int(inputs[0])
e = int(inputs[1])
nodes_shape = inputs[2].replace("\n","").split(" ")
connections = []
for i in range(3,len(inputs)):
    connections.append(inputs[i].replace("\n","").split(" ")[:2])
    
answer = [9,1,7,6,8,3,5,2,4]

edges = []
for i in range(len(nodes_shape)):
    edges.append([])
for edge in connections:
        edges[int(edge[0])].append(int(edge[1]))
        edges[int(edge[1])].append(int(edge[0]))
            
assignment = []
for i in range(len(nodes_shape)):
    assignment.append([nodes_shape[i],None,edges[i]])

domain_values = []
for i in range(len(nodes_shape)):
    domain_values.append([1,2,3,4,5,6,7,8,9])

In [4]:
for i in assignment:
    print(i)

['C', None, [1]]
['P', None, [0, 2]]
['P', None, [1, 3]]
['H', None, [2, 4]]
['T', None, [3, 5]]
['S', None, [4, 6]]
['S', None, [5, 7]]
['H', None, [6, 8]]
['P', None, [7, 9]]
['T', None, [8, 10]]
['S', None, [9, 11]]
['S', None, [10, 12]]
['H', None, [11, 13]]
['H', None, [12, 14]]
['S', None, [13, 15]]
['S', None, [14, 16]]
['H', None, [15, 17]]
['H', None, [16, 18]]
['T', None, [17, 19]]
['P', None, [18, 20]]
['P', None, [19, 21]]
['H', None, [20, 22]]
['T', None, [21, 23]]
['S', None, [22, 24]]
['S', None, [23, 25]]
['H', None, [24, 26]]
['P', None, [25, 27]]
['T', None, [26, 28]]
['S', None, [27, 29]]
['S', None, [28, 30]]
['H', None, [29, 31]]
['H', None, [30, 32]]
['S', None, [31, 33]]
['S', None, [32, 34]]
['H', None, [33, 35]]
['H', None, [34, 36]]
['T', None, [35, 37]]
['C', None, [36]]


In [2]:
def is_consistant(assingnment,index,value):
    for neighbour in assingnment[index][2]:
        if assingnment[int(neighbour)][1]==None:
            return True
    if assingnment[index][0] == "T": # triangel
        result = 1
        for neighbour in assingnment[index][2]:
            result *= assingnment[int(neighbour)][1]
        result = int(str(result)[0])
        if result != value:
            return False
    elif assingnment[index][0] == "S": #square
        result = 1
        for neighbour in assingnment[index][2]:
            result *= assingnment[int(neighbour)][1]
        result = str(result)
        result = int(result[len(result)-1])
        if result != value:
            return False
    elif assingnment[index][0] == "P": # pangZeli
        result = 0
        for neighbour in assingnment[index][2]:
            result += assingnment[int(neighbour)][1]
        result = int(str(result)[0])
        if result != value:
            return False
    elif assingnment[index][0] == "H": #shishZeli
        result = 0
        for neighbour in assingnment[index][2]:
            result += assingnment[int(neighbour)][1]
        result = str(result)
        result = int(result[len(result)-1])
        if result != value:
            return False
    elif assingnment[index][0] == "C": # Circle
        return True
    else:
        print("wrong input")
        return None
    return True
#______________________________________________________
def check(assignment):
    for i in range(len(assignment)):
        if assignment[i][1] == None:
            break
        if not is_consistant(assignment,i,assignment[i][1]):
            return False
    return True
#______________________________________________________
def solve(assignment,domain_values,MRV_heuristic,forward_checking):
    if(forward_checking):
        return back_track_fc_MRV(assignment,0,domain_values,MRV_heuristic)
    else:
        return back_track(assignment,0,domain_values[0])
#______________________________________________________
def back_track(assignment,node_index,domain_values):
    if(node_index >= len(assignment)):
        return True,[i[1] for i in assignment]
    for value in domain_values:
        assignment[node_index][1] = value
        if check(assignment):
            is_goal,answer = back_track(assignment.copy(),node_index+1,domain_values)
            if is_goal:
                return is_goal,answer
    return False,[]
#______________________________________________________
def forward_check(node_index,assignment,domain_values):
    for neighbour in assignment[node_index][2]:
        pop_list = []
        for value in domain_values[neighbour]:
            if not exists_sol(neighbour,assignment,value,domain_values):
                    pop_list.append(domain_values[neighbour].index(value))
        for i in sorted(pop_list,reverse=True):
            domain_values[neighbour].pop(i)
        if len(domain_values[neighbour])==0:
            return False,None
    return True,domain_values
#______________________________________________________
def exists_sol(node_index,assignment,node_value,domain_values):
    unassigned_list = []
    for neighbour in assignment[node_index][2]:
        if assignment[neighbour][1]==None:
            unassigned_list.append(neighbour)
    #########################################################
    def dfs(main_node_index,current_n,assignment,n_list):
        if current_n>=len(n_list):
            return is_consistant(assignment,main_node_index,node_value)
        for value in domain_values[n_list[current_n]]:
            assignment[n_list[current_n]][1] = value
            answer = dfs(main_node_index,current_n+1,assignment.copy(),n_list)
            if answer:
                return True
        return False
    #########################################################
    return dfs(node_index,0,assignment.copy(),unassigned_list)
#______________________________________________________
def MRV(assignment,domain_values):
    candidate_nodes = []
    for i in range(len(assignment)):
        if assignment[i][1]==None:
            candidate_nodes.append(i)
    if len(candidate_nodes)==0:
        return 0
    temp = []
    for i in candidate_nodes:
        temp.append(len(domain_values[i]))
    return candidate_nodes[temp.index(min(temp))]
#______________________________________________________
def back_track_fc_MRV(assignment,node_index,domain_values,MRV_heuristic):
    if None not in [i[1] for i in assignment]:
        return True,[i[1] for i in assignment]
    for value in domain_values[node_index]:
        assignment[node_index][1] = value
        if check(assignment):
            not_empty,new_domain_values = forward_check(node_index,assignment.copy(),copy.deepcopy(domain_values))
            if not not_empty:
                continue
            if MRV_heuristic:
                next_node = MRV(assignment,new_domain_values)
            else:
                next_node = node_index+1
            is_goal,answer = back_track_fc_MRV(assignment.copy(),next_node,new_domain_values,MRV_heuristic)
            if is_goal:
                return is_goal,answer
    return False,[]

In [3]:
start = time.time()
is_goal,answer = solve(np.copy(assignment),domain_values,MRV_heuristic=True,forward_checking=True)
end = time.time()
print("it took : "+ str(end - start) + " seconds")
print(answer)

it took : 0.2240900993347168 seconds
[5, 6, 1, 4, 3, 8, 6, 2, 6, 4, 8, 2, 4, 2, 8, 4, 3, 9, 6, 7, 1, 4, 3, 8, 6, 2, 6, 4, 8, 2, 4, 2, 8, 4, 3, 9, 6, 7]


In [13]:
t = [1,2]
x = [22,4,5,6]
i,m = min(x)

TypeError: cannot unpack non-iterable int object

In [434]:
print(domain_values)

[[1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9]]


In [37]:
[5, 6, 1, 4, 3, 8, 6, 2, 6, 4, 8, 2, 4, 2, 8, 4, 3, 9, 6, 7, 1, 4, 3, 8, 6, 2, 6, 4, 8, 2, 4, 2, 8, 4, 3, 9, 6, 7]

[2, 3, 4, 5, 6, 7]
